In [761]:
# import pandas and create original Database
import pandas as pd

In [762]:
csvfile = "Resources/purchase_data.csv"

In [763]:
purchase_data = pd.read_csv(csvfile)

In [764]:
frame_pd = pd.DataFrame(purchase_data)


In [765]:
#count lenght of unique database on SN to get the total number of players
frame_pd_output = frame_pd.copy()
frame_pd_output["Total Players"] = len(frame_pd["SN"].unique())
players_df = pd.DataFrame(frame_pd_output["Total Players"].head(1))
players_df


,Total Players
0,576


In [766]:
#use the dataframe above to calculate other summary level information
frame_pd_output["Number of Unique Items"] = len(frame_pd_output["Item ID"].unique())
frame_pd_output["Average_Price"] = frame_pd_output["Price"].mean()
#reformat the data to include $
frame_pd_output["Average Price"] = frame_pd_output["Average_Price"].astype(float).map("${:,.2f}".format)
frame_pd_output["Number of Purchases"] = frame_pd_output["Purchase ID"].count()
frame_pd_output["Total_Revenue"] = frame_pd_output["Price"].sum()
#reformat the data to include $
frame_pd_output["Total Revenue"] = frame_pd_output["Total_Revenue"].astype(float).map("${:,.2f}".format)
statistics_df = pd.DataFrame(frame_pd_output[["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"]].head(1))
statistics_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [767]:
#create a new dataframe to analyze gender
unique_df = frame_pd_output[["SN","Gender","Age"]]
#remove duplicate lines so you count individual users by gender
unique_df = unique_df.drop_duplicates()
gender_df = pd.DataFrame(unique_df["Gender"].value_counts())
gender_df["Percentage_of_Players"] = pd.DataFrame(unique_df["Gender"].value_counts("p")*100)
gender_df["Percentage of Players"] = gender_df["Percentage_of_Players"].astype(float).map("{:.2f}%".format)
#remane columns to match expected output
gender_df[["Gender","Percentage of Players"]].rename(columns={"Gender":"Total Count"})


,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [768]:
#groupby gender and count the unique number of entires per gender
#use mean and sum to get the total price information
frame_pd_gender = frame_pd.groupby(["Gender"])
purchase_count = frame_pd_gender["Purchase ID"].count().head()

avg_pp = frame_pd_gender["Price"].mean()
avg_pp_f = avg_pp.astype(float).map("${:,.2f}".format)
tot_pp = frame_pd_gender["Price"].sum()
tot_pp_f = tot_pp.astype(float).map("${:,.2f}".format)
avg_tot_pp = tot_pp / gender_df.loc[:,"Gender"]
avg_tot_pp_f = avg_tot_pp.astype(float).map("${:,.2f}".format)

#populate the columns with the variables calculated above
gender_output = pd.DataFrame({"Purchase Count": purchase_count,
                            "Average Purchase Price": avg_pp_f,
                             "Total Purchase Value": tot_pp_f,
                             "Avg Total Purchase per Person":avg_tot_pp_f})
gender_output

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [769]:
#create bins to hold different age ranges - use the maximum age as the top bin so that all individuals are accounted for
max_age = frame_pd["Age"].max()
bins = [0,9,14,19,24,29,34,39,max_age]
bin_ranges = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
#use pd.cut to put the data into bins and then groupby age group to export the bins to the summary table
unique_df["Age Group"] = pd.cut(unique_df["Age"],bins, labels=bin_ranges)
bin_df = unique_df.groupby("Age Group")
bin_df_len = len(unique_df)
bin_df = pd.DataFrame(bin_df["Age Group"].count())
bin_df["Percentage_of_Players"] = bin_df/bin_df_len*100
bin_df["Percentage of Players"] = bin_df["Percentage_of_Players"].astype(float).map("{:.2f}%".format)
bin_df = bin_df.rename(columns={"Age Group":"Total Count"})
bin_df[["Total Count","Percentage of Players"]]



,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [770]:
# use bins again to create a new summary table to include additional data (Price)

max_age = frame_pd["Age"].max()
bins = [0,9,14,19,24,29,34,39,max_age]
bin_ranges = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
age_summary = frame_pd

age_summary["Age Group"] = pd.cut(age_summary["Age"],bins, labels=bin_ranges)
age_summary_gb = age_summary.groupby("Age Group")

# create different data frames to hold each set of data needed in the final output
age_summary_gb1 = pd.DataFrame(age_summary_gb["Purchase ID"].count())
age_summary_gb2 = pd.DataFrame(age_summary_gb["Price"].mean())
age_summary_gb3 = pd.DataFrame(age_summary_gb["Price"].sum())
age_summary_gb4 = pd.DataFrame(age_summary_gb["Price"].sum()/bin_df["Total Count"])
# merge the data frames together based off of the age group
merge_df = pd.merge(age_summary_gb1,age_summary_gb2, on="Age Group")
merge_df = pd.merge(merge_df,age_summary_gb3, on="Age Group")
merge_df = pd.merge(merge_df,age_summary_gb4, on="Age Group")
merge_df = merge_df.rename(columns={"Purchase ID":"Purchase Count",
                                  "Price_x":"Average Purchase Price",
                                  "Price_y":"Total Purchase Value",
                                  0:"Avg Total Purchase per Person"})
# reformat and print the dataframes
merge_df["Average Purchase Price"] =merge_df["Average Purchase Price"].astype(float).map("${:,.2f}".format)
merge_df["Total Purchase Value"] =merge_df["Total Purchase Value"].astype(float).map("${:,.2f}".format)
merge_df["Avg Total Purchase per Person"] =merge_df["Avg Total Purchase per Person"].astype(float).map("${:,.2f}".format)
merge_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [771]:
# create new dataframe to house top spender information
top_spender_df = frame_pd
top_spender_df_gb = top_spender_df.groupby("SN")
# create and merge new data frames on the SN column to get the top spender information
top_spender_gb1= pd.DataFrame(top_spender_df_gb["SN"].count())
top_spender_gb1 = top_spender_gb1.rename(columns={'SN':'ScreenName'})
top_spender_gb2= pd.DataFrame(top_spender_df_gb["Price"].mean())
#sort by top spender - sum of Price
top_spender_gb3= pd.DataFrame(top_spender_df_gb["Price"].sum().sort_values(ascending = False))
merge_df_ts = pd.merge(top_spender_gb1,top_spender_gb2, on="SN")
merge_df_ts = pd.merge(merge_df_ts,top_spender_gb3, on="SN")
merge_df_ts = merge_df_ts.rename(columns={"ScreenName":"Purchase Count",
                                         "Price_x":"Average Purchase Price",
                                         "Price_y":"Total Purchase Value"})
merge_df_ts = pd.DataFrame(merge_df_ts[["Purchase Count","Average Purchase Price","Total Purchase Value"]].sort_values(by="Total Purchase Value",ascending = False))
merge_df_ts["Average Purchase Price"]= merge_df_ts["Average Purchase Price"].astype(float).map("${:,.2f}".format)
merge_df_ts["Total Purchase Value"]= merge_df_ts["Total Purchase Value"].astype(float).map("${:,.2f}".format)
#export top 5 spenders
merge_df_ts.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [772]:
pop_item_df = frame_pd
#group new dataframe by item Id and Item Name
pop_item_df = pop_item_df.groupby(["Item ID","Item Name"])
#create and merge data frames to match the output
pop_item_gb1 = pd.DataFrame(pop_item_df["Item ID"].count())
pop_item_gb1 = pop_item_gb1.rename(columns={"Item ID":"Item_ID"})
pop_item_gb2 = pd.DataFrame(pop_item_df["Price"].mean())
pop_item_gb3 = pd.DataFrame(pop_item_df["Price"].sum())
merge_pop_df = pd.merge(pop_item_gb1,pop_item_gb2,on=["Item ID","Item Name"])
merge_pop_df = pd.merge(merge_pop_df,pop_item_gb3,on=["Item ID","Item Name"])
merge_pop_df = merge_pop_df.rename(columns={"Item_ID":"Purchase Count",
                                         "Price_x":"Item Price",
                                         "Price_y":"Total Purchase Value"})
#reformat data to show with $
#sort data by Purchase Count to get the top 5
merge_pop_df = pd.DataFrame(merge_pop_df[["Purchase Count","Item Price","Total Purchase Value"]].sort_values(by="Purchase Count",ascending = False))
merge_pop_df["Item Price"]= merge_pop_df["Item Price"].astype(float).map("${:,.2f}".format)
merge_pop_df["Total Purchase Value"]= merge_pop_df["Total Purchase Value"].astype(float).map("${:,.2f}".format)
#sort data by Item Id - this gives you the top 5 items in terms of purchase count and in order of decending ID
#While this matches the assignment, I noticed a few things about the data that seemed wrong here
#First, the item Final Critic had two Ids so it didn't get picked up on this list
# second, it simple sorted the purchase count and the price is sorted from smallest to larges. this means that all of the other items that had 8 purchases probably should have been higher on the list than the item ID 19
merge_pop_df.head().sort_values(by="Item ID", ascending = False)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [773]:
#create new dataframe to ensure the original doesn't get messed up
merge_pop_df2 = merge_pop_df
#recalculate the total purchase value to ensure it is the correct data type
merge_pop_df2["Total Purchase Value"] = pd.DataFrame(pop_item_df["Price"].sum())
#sort based off of the Total Purchase Value
merge_pop_df2 = merge_pop_df2.sort_values(by="Total Purchase Value",ascending = False)
merge_pop_df2["Total Purchase Value"]= merge_pop_df2["Total Purchase Value"].astype(float).map("${:,.2f}".format)
merge_pop_df2.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
